<a href="https://colab.research.google.com/github/edegan-hering/RPA_Challenge/blob/main/rpachallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este script automatiza a instalação do Google Chrome em ambientes Linux baseados no Debian (como Ubuntu ou Google Colab)

In [1]:
# Atualiza os pacotes
!apt-get update
# Instala dependência necessária para o Chrome
!apt-get install -y libvulkan1
# Baixa o instalador do Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# Tenta instalar o Chrome, e corrige dependências se necessário
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -f install -y
# Remove o instalador para limpar
!rm google-chrome-stable_current_amd64.deb

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,561 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,347 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd

Instala as bibliotecas Python necessárias para automação com Selenium e para gerenciar automaticamente o ChromeDriver

In [2]:
!pip install selenium chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

Importa bibliotecas necessárias para automação


In [3]:
# Detecta e instala automaticamente a versão correta do ChromeDriver
import chromedriver_autoinstaller
# Biblioteca principal para controle do navegador via Selenium
from selenium import webdriver
# Para localizar elementos na página (ex: By.ID, By.XPATH)
from selenium.webdriver.common.by import By
# Para pausas/sleep no código
import time
# Para fazer requisições HTTP (ex: baixar dados)
import requests
# Manipulação e análise de dados em tabelas (DataFrame)
import pandas as pd

Instala automaticamente o ChromeDriver compatível e configura o navegador


In [4]:
# Instala automaticamente o ChromeDriver compatível com o Chrome instalado
chromedriver_autoinstaller.install()

options = webdriver.ChromeOptions()
# Executa o Chrome em modo headless (sem interface gráfica)
options.add_argument('--headless')
# Necessário em ambientes Linux restritos, como containers
options.add_argument('--no-sandbox')
 # Evita erros de uso de memória compartilhada
options.add_argument('--disable-dev-shm-usage')

# Inicializa o navegador Chrome com as opções definidas
driver = webdriver.Chrome(options=options)


Abre o site do RPA Challenge, encontra e baixa o arquivo Excel

In [5]:
# Abre a página e espera 2 segundos para garantir carregamento
driver.get('https://rpachallenge.com/')
time.sleep(2)

# Busca o link do arquivo Excel na página pelo atributo href que contenha ".xlsx"
download_link = driver.find_element(By.XPATH, "//a[contains(@href, '.xlsx')]").get_attribute("href")

# Faz o download do arquivo via requisição HTTP
response = requests.get(download_link)

# Salva o arquivo baixado no caminho especificado
with open("/content/challenge.xlsx", "wb") as f:
    f.write(response.content)

# Confirmação do download
print("Downloaded to /content/challenge.xlsx")



Downloaded to /content/challenge.xlsx


Lê os dados do Excel, inicia o desafio no site, preenche e envia o formulário

In [6]:
# Lê a planilha Excel para um DataFrame
df = pd.read_excel("challenge.xlsx")

# Clica no botão "Start" para iniciar o desafio
driver.find_element(By.XPATH, "//button[text()='Start']").click()

 # Para cada linha da tabela, preenche o formulário com os dados correspondentes
for index, row in df.iterrows():
    driver.find_element(By.XPATH, "//input[@ng-reflect-name='labelFirstName']").send_keys(row['First Name'])
    driver.find_element(By.XPATH, "//input[@ng-reflect-name='labelLastName']").send_keys(row['Last Name '])
    driver.find_element(By.XPATH, "//input[@ng-reflect-name='labelCompanyName']").send_keys(row['Company Name'])
    driver.find_element(By.XPATH, "//input[@ng-reflect-name='labelRole']").send_keys(row['Role in Company'])
    driver.find_element(By.XPATH, "//input[@ng-reflect-name='labelAddress']").send_keys(row['Address'])
    driver.find_element(By.XPATH, "//input[@ng-reflect-name='labelEmail']").send_keys(row['Email'])
    driver.find_element(By.XPATH, "//input[@ng-reflect-name='labelPhone']").send_keys(str(row['Phone Number']))

    # Envia o formulário
    driver.find_element(By.XPATH, "//input[@type='submit']").click()

    # Pequena pausa para evitar falhas no envio
    time.sleep(0.3)

# Captura a mensagem de resultado exibida no site
message = driver.find_element(By.CLASS_NAME, "message2").text

# Exibe o resultado final
print("Resultado:", message)

# Fecha o navegador
driver.quit()


Resultado: Your success rate is 100% ( 70 out of 70 fields) in 12028 milliseconds
